In [1]:
# 

In [2]:
# Can your fine-tuned LLM detect and explain unseen network failures?

In [3]:
!pip install -Uqq vllm==0.10.0 pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 995.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/57

In [4]:
%%writefile generate_traces.py

# =============================================
# Kaggle Notebook: Generate Reasoning Traces with vLLM (Qwen2.5-7B-Instruct)
# Accelerator: GPU T4 × 2
# =============================================

# ─── 1. Install dependencies ───────────────────────────────────────
# !pip install -Uqq vllm pandas tqdm

# ─── 2. Imports ─────────────────────────────────────────────────────
import pandas as pd
from pathlib import Path
import argparse
import time
from tqdm import tqdm
from vllm import LLM, SamplingParams

# ─── 3. Configuration ───────────────────────────────────────────────
SAVE_EVERY = 5           # Save progress every N samples
BATCH_SIZE = 16          # Adjust: 8–32 depending on memory (higher = faster)
                         # Start with 16; increase if no OOM

# Use AWQ-quantized version for speed + lower memory (strongly recommended)
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct-AWQ"
# Alternative (slower, more memory): "Qwen/Qwen2.5-7B-Instruct"

MAX_TOKENS = 1600
TEMPERATURE = 0.2
MAX_MODEL_LEN = 8192

# ─── 4. System Prompt (unchanged) ──────────────────────────────────
SYSTEM_PROMPT = """You are an expert 5G telecommunications network analyst specializing in root cause analysis (RCA) of throughput degradation issues.
You will be given:

1. A QUESTION containing a real 5G drive-test scenario with:   
   - User plane data (throughput, RSRP, SINR, handovers, etc.)   
   - Engineering parameters (cell configs, downtilt, azimuth, power, etc.)   
   - Eight candidate root causes (C1-C8)

2. The CORRECT ANSWER (one of C1-C8)

Your task is to generate a structured reasoning trace that explains WHY the correct answer is right, following this exact 3-task structure:

**Task 1: Data Analysis** Systematically analyze the provided data:

* Throughput patterns: When does throughput drop? What are the values?
* Serving cell analysis: PCI, RSRP, SINR, coverage distance, configuration
* Neighbor cell analysis: Top neighbors, their signal strength, throughput comparison
* Mobility patterns: Handovers, GPS speed, location changes
* Resource allocation: Scheduled RBs (Resource Blocks)
* Cell configurations: Downtilt angles, azimuth, beam scenarios, PCI mod 30 relationships
* Coverage topology: Distances, overlapping coverage, co-location

**Task 2: Root Cause Analysis** Use systematic elimination or contradiction-based reasoning:

* For EACH of the 8 candidate root causes (C1-C8), evaluate whether it fits the observed data
* Rule out implausible causes by showing contradictions with the data
* For causes that cannot be ruled out, explain what evidence supports them
* Compare the strength of evidence for remaining candidates

Evaluation guidelines:
* C1 (Excessive downtilt): Check total downtilt angle vs. beam scenario vertical beamwidth
* C2 (Over-shooting): Check if serving cell distance exceeds 1km
* C3 (Higher neighbor throughput): Compare serving vs. neighbor cell throughput
* C4 (Overlapping coverage): Check for non-colocated co-frequency neighbors
* C5 (Frequent handovers): Count handover events
* C6 (PCI mod 30 conflict): Calculate PCI mod 30 for serving and neighbors
* C7 (High speed): Check if GPS speed exceeds 40 km/h
* C8 (Low RBs): Check if average scheduled RBs are below 160

**Task 3: Conclusion**
* Identify the most plausible root cause based on Task 2 analysis
* Explain why this cause best explains the observed throughput degradation
* State the final answer clearly (e.g., "C3")

**Summary** Provide a concise 2-3 sentence summary of the root cause and key evidence.

IMPORTANT FORMATTING:
* Use clear section headers: "Task 1: Data Analysis", "Task 2: Root Cause Analysis", "Task 3: Conclusion", "Summary"
* Be specific with numbers and measurements from the data
* Show your reasoning step-by-step
* The final answer MUST match the CORRECT ANSWER provided
* Focus on explaining WHY the correct answer is right, not on discovering it

Your reasoning should be thorough, technical, and grounded in the actual measurements and configurations provided in the question.
"""

# ─── 5. Validation function (unchanged) ─────────────────────────────
def validate_reasoning_trace(trace: str, answer: str) -> bool:
    return all([
        "Task 1" in trace or "Data Analysis" in trace,
        "Task 2" in trace or "Root Cause Analysis" in trace,
        "Task 3" in trace or "Conclusion" in trace,
        "Summary" in trace,
        answer in trace,
        len(trace) > 200
    ])

# ─── 6. Batched inference function ──────────────────────────────────
def generate_batch(llm, prompts, sampling_params, batch_size=BATCH_SIZE):
    results = []
    for i in tqdm(range(0, len(prompts), batch_size), desc="Inference batches"):
        batch = prompts[i:i + batch_size]
        try:
            outputs = llm.generate(batch, sampling_params)
            batch_results = [out.outputs[0].text.strip() for out in outputs]
            results.extend(batch_results)
        except Exception as e:
            print(f"Batch error: {e}")
            results.extend([""] * len(batch))
    return results

# ─── 7. Main generation logic ───────────────────────────────────────
def generate_reasoning(input_path: str | Path, debug: bool = False, validate: bool = True):
    input_path = Path(input_path)
    output_path = Path("/kaggle/working/augmented_train_with_trace.csv")

    # Load or resume
    if output_path.exists():
        print(f"Resuming from: {output_path}")
        df = pd.read_csv(output_path, on_bad_lines='skip')
    else:
        print(f"Starting fresh from: {input_path}")
        df = pd.read_csv(input_path, on_bad_lines='skip')
        if 'reasoning_trace' not in df.columns:
            df['reasoning_trace'] = ""

    # Pending rows
    pending_mask = df['reasoning_trace'].isna() | (df['reasoning_trace'].str.strip() == "")
    pending_df = df[pending_mask].copy()

    if debug:
        pending_df = pending_df.head(5)
        print("DEBUG MODE: processing first 5 rows only")

    if len(pending_df) == 0:
        print("All rows already processed.")
        return

    print(f"Remaining questions to process: {len(pending_df)}")

    # ─── Load vLLM ──────────────────────────────────────────────────
    print(f"Loading model: {MODEL_NAME} ... (may take 3–8 minutes)")
    llm = LLM(
        model=MODEL_NAME,
        tensor_parallel_size=2,           # Use both T4 GPUs
        gpu_memory_utilization=0.92,      # Slightly under 1.0 to avoid OOM
        max_model_len=MAX_MODEL_LEN,
        dtype="auto",                     # Usually bfloat16 or float16
        enforce_eager=True,               # Sometimes more stable on T4
        # quantization="awq",             # already quantized model → no need
    )

    sampling_params = SamplingParams(
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        skip_special_tokens=True,
    )

    # Prepare prompts
    prompts = []
    for _, row in pending_df.iterrows():
        user_content = f"QUESTION: {row['question']}\nCORRECT ANSWER: {row['answer']}"
        full_prompt = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{user_content}<|im_end|>\n<|im_start|>assistant\n"
        prompts.append(full_prompt)

    # Generate
    print("Starting batched generation...")
    reasonings = generate_batch(llm, prompts, sampling_params)

    # Write back + validate
    valid_count = invalid_count = 0
    processed_since_save = 0

    for idx, reasoning in zip(pending_df.index, reasonings):
        if reasoning:
            df.at[idx, 'reasoning_trace'] = f"<thinking>{reasoning}</thinking>"
            if validate:
                if validate_reasoning_trace(reasoning, df.at[idx, 'answer']):
                    valid_count += 1
                else:
                    invalid_count += 1
                    print(f"⚠️ Row {idx} validation failed (may need manual check)")
        processed_since_save += 1

        if processed_since_save >= SAVE_EVERY:
            df.to_csv(output_path, index=False)
            print(f"💾 Saved progress after {processed_since_save} samples")
            processed_since_save = 0

    # Final save
    df.to_csv(output_path, index=False)

    # Summary
    print("\n" + "="*60)
    print(f"Done! Output saved to: {output_path}")
    print(f"Processed this run : {len(pending_df)} samples")
    if validate:
        print(f"Valid traces       : {valid_count}")
        print(f"Invalid traces     : {invalid_count}")
    print("="*60)

# ─── 8. Run ─────────────────────────────────────────────────────────
INPUT_CSV = "/kaggle/input/the-ai-telco-troubleshooting-challenge-dataset/augmented_train.csv"

if __name__ == "__main__":
    generate_reasoning(
        input_path=INPUT_CSV,
        debug=False,           # Change to True for quick testing on 5 samples
        validate=True
    )

Writing generate_traces.py


In [5]:
!python generate_traces.py

2026-01-27 06:31:42.947515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769495503.159506      85 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769495503.221824      85 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769495503.750601      85 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769495503.750644      85 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769495503.750652      85 computation_placer.cc:177] computation placer alr

In [6]:
# https://arxiv.org/pdf/2512.19530
# https://claude.ai/chat/0c3645d3-8dbf-483d-9d0c-b6dc98af9771

In [7]:
# 